In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D,Flatten,Dense,Activation,Add,BatchNormalization,MaxPooling2D,AveragePooling2D,Input,ZeroPadding2D
from keras.initializers import glorot_uniform
from keras.models import Model
import keras
from keras.regularizers import l2
(X_train,y_train),(X_test,y_test)=keras.datasets.cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 12s 0us/step


In [0]:
def identity_block(X,f,filters):
    
    X_temp = X
    F1,F2 = filters
    
    X = Conv2D(F1,(f,f), strides=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [0]:
def conv_block(X,f,filters,s):
    X_temp = X
    F1,F2 = filters
    
    X_temp=Conv2D(F2,(1,1), strides=(s,s), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X_temp)
    X_temp=BatchNormalization(axis=3)(X_temp)
    
    X = Conv2D(F1,(f,f), strides=(s,s),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [0]:
def ModelCustom(input_shape,classes):
    
    X_input=Input(input_shape)
    X=ZeroPadding2D((1,1))(X_input)
    X=Conv2D(32,(3,3), strides=(1,1), kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    #X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    X=conv_block(X,3,(32,32),1)
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    
    X=conv_block(X,3,(64,64),2)
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    
    X=conv_block(X,3,(128,128),2)
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    
    #X=conv_block(X,3,(256,256),2)
    #X=identity_block(X,3,(256,256))
    
    
    #X=conv_block(X,3,(512,512),2)    #commented to reduce params
    #X=identity_block(X,3,(512,512))  # commented out to reduce params
    
    
    X = AveragePooling2D((8, 8))(X)
    
    X = Flatten()(X)
    
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0),kernel_regularizer=l2(1e-4))(X)
    
    model=Model(inputs = X_input, outputs = X, name='ResNet34')
    
    return model

In [0]:
model = ModelCustom((32,32,3),10)

In [6]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   896         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
___________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [0]:
image_gen= keras.preprocessing.image.ImageDataGenerator( 
    horizontal_flip=True,
    zca_epsilon=1e-06,
    fill_mode="nearest",
    width_shift_range=0.1,
    height_shift_range=0.1)

image_gen.fit(X_train)

In [10]:
model.fit_generator(image_gen.flow(X_train, y_train, batch_size=128),
                        validation_data=(X_test, y_test),
                        epochs=30,workers=4)

Epoch 1/30
391/391 [==============================] - 153s 392ms/step - loss: 1.8087 - accuracy: 0.4891 - val_loss: 3.0849 - val_accuracy: 0.3303
Epoch 2/30
391/391 [==============================] - 138s 353ms/step - loss: 1.2993 - accuracy: 0.6669 - val_loss: 1.2883 - val_accuracy: 0.6763
Epoch 3/30
391/391 [==============================] - 138s 354ms/step - loss: 1.0872 - accuracy: 0.7336 - val_loss: 1.3817 - val_accuracy: 0.6391
Epoch 4/30
391/391 [==============================] - 138s 354ms/step - loss: 0.9642 - accuracy: 0.7706 - val_loss: 1.3348 - val_accuracy: 0.6717
Epoch 5/30
391/391 [==============================] - 138s 354ms/step - loss: 0.8736 - accuracy: 0.7958 - val_loss: 1.2558 - val_accuracy: 0.6854
Epoch 6/30
391/391 [==============================] - 138s 353ms/step - loss: 0.8190 - accuracy: 0.8099 - val_loss: 1.0026 - val_accuracy: 0.7393
Epoch 7/30
391/391 [==============================] - 138s 353ms/step - loss: 0.7588 - accuracy: 0.8282 - val_loss: 1.1488 -